In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from qat.qpus import get_default_qpu
from qat.core import Batch, Job

from qat.fermion.transforms import transform_to_jw_basis
from qat.fermion.hamiltonians import make_embedded_model
from qat.fermion.circuits import make_shallow_circ, make_ldca_circ

from qat.fermion.chemistry.pyscf_tools import perform_pyscf_computation
from qat.fermion.chemistry import MolecularHamiltonian, MoleculeInfo
from qat.fermion.trotterisation import make_trotterisation_routine
from qat.fermion.chemistry.ucc import construct_ucc_ansatz, guess_init_params, get_hf_ket, get_cluster_ops
from qat.qpus import get_default_qpu

from qat.plugins import ScipyMinimizePlugin, MultipleLaunchesAnalyzer

In [ ]:
from openvqe.common_files import molecule_factory

In [ ]:
from qat.fermion.chemistry.pyscf_tools import perform_pyscf_computation

geometry = [("H", (0.0, 0.0, 0.0)), ("H", (0.0, 0.0, 0.7414))]
basis = "sto-3g"
spin = 0
charge = 0

(
    rdm1,
    orbital_energies,
    nuclear_repulsion,
    n_electrons,
    one_body_integrals,
    two_body_integrals,
    info,
) = perform_pyscf_computation(geometry=geometry, basis=basis, spin=spin, charge=charge, run_fci=True)

print(
    f" HF energy :  {info['HF']}\n",
    f"MP2 energy : {info['MP2']}\n",
    f"FCI energy : {info['FCI']}\n",
)
print(f"Number of qubits before active space selection = {rdm1.shape[0] * 2}")

nqbits = rdm1.shape[0] * 2
print("Number of qubits = ", nqbits)

In [ ]:
mol_h = MolecularHamiltonian(one_body_integrals, two_body_integrals, nuclear_repulsion)
H = mol_h.get_electronic_hamiltonian()

In [ ]:
from qat.fermion.transforms import transform_to_jw_basis, transform_to_parity_basis, transform_to_bk_basis

# Using the Jordan-Wigner transform
hamiltonian_sp = transform_to_jw_basis(H)
# Define the initial Hartree-Fock state
ket_hf_init = get_hf_ket(n_electrons, nqbits=nqbits)

# Compute the cluster operators
cluster_ops = get_cluster_ops(n_electrons, nqbits=nqbits)
cluster_ops_sp = [transform_to_jw_basis(t_o) for t_o in cluster_ops]
model = hamiltonian_sp
print("This model has the number of qubit:", model.nbqbits)
print("The number of excitation for this model", len(cluster_ops_sp))
print(model)


In [ ]:
model_matrix_sp = model.get_matrix(sparse=True)
from scipy.sparse.linalg import eigsh
eigval, eigvec = eigsh(model_matrix_sp, k=13)
print(eigval)
#print(eigvec)


In [ ]:
# To write circuits
from qat.lang.AQASM import *
from qat.lang import Program
# To define an observable
from qat.core import Observable, Term
# our Plugin
from qat.plugins import ObservableSplitter
# and a QPU
from qat.qpus import get_default_qpu
from qat.interop.qiskit import qlm_to_qiskit

In [ ]:
import matplotlib.pyplot as plt
from numpy import binary_repr
from qat.lang import Program, X
from qat.qpus import get_default_qpu

# Assuming hamiltonian_sp is defined somewhere in your code
# and box_list contains the values you mentioned
box_list = [0, 1, 2, 4, 8, 3, 5, 9, 6, 10, 12, 7, 11, 13, 14, 15]


def create_program(value, nqbits):
    ket_value = binary_repr(value)
    padded_ket_value = ket_value.zfill(nqbits)  # Pad with zeros to the left
    list_ket_value = [int(c) for c in padded_ket_value]

    prog = Program()
    qb = prog.qalloc(nqbits)

    # Apply X gates based on the binary representation of the current value
    for j in range(min(nqbits, len(list_ket_value))):
        if list_ket_value[j] == 1:
            prog.apply(X, qb[j])

    return prog


excitation_values = []
box_states = []

for value in box_list:
    nqbits = model.nbqbits
    prog = create_program(value, nqbits)

    circ = prog.to_circ()
    qpu = get_default_qpu()

    # Assuming hamiltonian_sp is defined somewhere in your code
    job = circ.to_job(job_type="OBS", observable=model)
    res = qpu.submit(job)

    box_states.append(binary_repr(value, width=nqbits))
    excitation_values.append(res.value)

    # Print the box state and the result for each value
    print(f"Box State: {binary_repr(value, width=nqbits)}, Value: {value}, Excitation: {res.value}")

# Sorting the data by excitation values
sorted_data = sorted(zip(box_states, excitation_values), key=lambda x: x[1])

# Extracting sorted box states and excitation values
sorted_box_states, sorted_excitation_values = zip(*sorted_data)

# Plotting histogram with a larger figure size, crimson color, and a different style
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
bars = plt.bar(sorted_box_states, sorted_excitation_values, color='crimson', alpha=0.7, edgecolor='black', linestyle='-', linewidth=1.2)
plt.xlabel('Box State')
plt.ylabel('Excitation')
plt.title('Excitation Histogram by Box State (Increasing Order)')
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add text annotations on top of each bar
for bar, excitation in zip(bars, sorted_excitation_values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.005, f'{excitation:.4f}', ha='center', va='bottom')

plt.tight_layout()  # Adjust layout for better spacing
plt.show()

From this graph, I decided the three exact values 
-  Ground state 1100 with k=12 
- The first excited state 0100 with k=4 
- The second excited state 1101 with k=13
- The third excited state 0110 with k= 6

#### Step 1: Draw the circuit $U(\theta)$

This circuit is get inspiration the fact that instead of the CNOT we have the Z control gate 

In [ ]:
from qat.lang.AQASM import Program, QRoutine, RY, CNOT, RX, Z, H, RZ, I
from qat.core import Observable, Term, Circuit
from qat.lang.AQASM.gates import Gate
import matplotlib as mpl
import numpy as np
from typing import Optional, List
import warnings

In [ ]:
#With initial state
def U_theta(
    nqbits: int,
    theta: List[float],
    k_init: int,
    n_layers: int = 1,
    rotation_gates: List[Gate] = None,
    entangling_gate: Gate = CNOT,
) -> Circuit: #linear entanglement

    prog = Program()
    reg = prog.qalloc(nqbits)
    
    
    state = binary_repr(k_init)
    state_pad = state.zfill(nqbits)
    state_lst = [int(c) for c in state_pad]


    for j in range(min(nqbits, len(state_lst))):
        if int(state_lst[j] == 1):
            prog.apply(X, reg[j])
        
        if int(state_lst[j] == 0):
            prog.apply(I, reg[j])
    
    print("This initial state is ", state_pad)
    
    
    
    
    if rotation_gates is None:
        rotation_gates = [RZ]

    n_rotations = len(rotation_gates)

    
    
    ind_theta = 0


    
    for i in range(nqbits):

        for rot in rotation_gates:

            prog.apply(rot(theta[ind_theta]), reg[i])
            ind_theta += 1
    
    for k in range(n_layers):


        for i in range(nqbits - 1):
            prog.apply(CNOT, reg[i], reg[i+1])
            
        for i in range(nqbits):
            for rot in rotation_gates:
                            
                prog.apply(rot(theta[ind_theta]), reg[i])
                ind_theta += 1
    

    return prog.to_circ()

In [ ]:
import numpy as np
n_rotations = 2
nqbits =4
n_layers = 3
k= 9

theta_0 = [np.random.uniform(0, 2*np.pi) for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_layers))]
k_lst = [12,4,13]
theta = [prog.new_var(float, rf"\theta_{{{i}}}") for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_layers))]

#Circuit implementation with applied state
circ_ground = U_theta(4, theta, 12, 3, [RX,RZ], CNOT)
circ_ground.display()

circ_exci_1 = U_theta(4, theta, 4, 3, [RX,RZ], CNOT)
circ_exci_1.display()

circ_exci_2 = U_theta(4, theta, 13, 3, [RX,RZ], CNOT)
circ_exci_2.display()


2B: Test with compact version

In [ ]:
import scipy
import numpy as np
from qat.vsolve.optimize import ScipyMinimizePlugin
from qat.qpus import get_default_qpu
import matplotlib.pyplot as plt
import scipy.optimize as sp_optimize

# Define your model, circuits, and other parameters
# model = ...
# circ_funct_ground_input, circ_funct_exci_1_input, circ_funct_exci_2_input, circ_funct_exci_3_input = ...

def opt_funct(circ_funct_ground_input, circ_funct_exci_1_input, circ_funct_exci_2_input, model, qpu, nqbits, energy_circ_ground, energy_circ_exci_1, energy_circ_exci_2):
    def input_funct(x):
        opt_circ_ground = circ_funct_ground_input.bind_variables({k: v for k, v in zip(sorted(circ_funct_ground_input.get_variables()), x)})
        opt_circ_exci_1 = circ_funct_exci_1_input.bind_variables({k: v for k, v in zip(sorted(circ_funct_exci_1_input.get_variables()), x)})
        opt_circ_exci_2 = circ_funct_exci_2_input.bind_variables({k: v for k, v in zip(sorted(circ_funct_exci_2_input.get_variables()), x)})
        stack0 = qpu.submit(opt_circ_ground.to_job(observable=model))
        stack1 = qpu.submit(opt_circ_exci_1.to_job(observable=model))
        stack2 = qpu.submit(opt_circ_exci_2.to_job(observable=model))

        
        energy_ground = stack0.value
        energy_exci_1 = stack1.value
        energy_exci_2 = stack2.value


        energy_circ_ground[method].append(energy_ground)
        energy_circ_exci_1[method].append(energy_exci_1)
        energy_circ_exci_2[method].append(energy_exci_2)


        energy_cost =  energy_ground + energy_exci_1 +  energy_exci_2 
        return energy_cost

    def callback(x):
        opt_circ_ground = circ_funct_ground_input.bind_variables({k: v for k, v in zip(sorted(circ_funct_ground_input.get_variables()), x)})
        opt_circ_exci_1 = circ_funct_exci_1_input.bind_variables({k: v for k, v in zip(sorted(circ_funct_exci_1_input.get_variables()), x)})
        opt_circ_exci_2 = circ_funct_exci_2_input.bind_variables({k: v for k, v in zip(sorted(circ_funct_exci_2_input.get_variables()), x)})

        
        stack0 = qpu.submit(opt_circ_ground.to_job(observable=model))
        stack1 = qpu.submit(opt_circ_exci_1.to_job(observable=model))
        stack2 = qpu.submit(opt_circ_exci_2.to_job(observable=model))

        
        energy_ground = stack0.value
        energy_exci_1 = stack1.value
        energy_exci_2 = stack2.value


        energy_circ_ground[method].append(energy_ground)
        energy_circ_exci_1[method].append(energy_exci_1)
        energy_circ_exci_2[method].append(energy_exci_2)
    def circuit(x):
        opt_circ_exci_2 = circ_funct_exci_2_input.bind_variables({k: v for k, v in zip(sorted(circ_funct_exci_2_input.get_variables()), x)})
        qpu = get_default_qpu()
        job = opt_circ_exci_2.to_job()
        resultnew = qpu.submit(job)
        opt_circ = opt_circ_exci_2.bind_variables(eval(resultnew.meta_data["parameter_map"]))
        return  opt_circ
    return input_funct, callback, circuit

# Initialize the QPU
qpu = get_default_qpu()

# Set the method
method = "BFGS"



# Initialize energy dictionaries
energy_circ_ground = {method: []}
energy_circ_exci_1 = {method: []}
energy_circ_exci_2 = {method: []}




input_funct,callback,circuit = opt_funct(circ_ground, circ_exci_1, circ_exci_2, model, qpu, nqbits, energy_circ_ground, energy_circ_exci_1, energy_circ_exci_2)
options = {"disp": True, "maxiter": 500, "gtol": 1e-7}


Optimizer = scipy.optimize.minimize(input_funct, x0=theta_0, method = "BFGS", callback=callback)


print(circuit)

In [ ]:

print(Optimizer)
print(Optimizer.x)

In [ ]:
print(energy_circ_ground)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(energy_circ_ground[method], label='Energy Circ Ground')
plt.plot(energy_circ_exci_1[method], label='Energy Circ Exci 1')
plt.plot(energy_circ_exci_2[method], label='Energy Circ Exci 2')
plt.xlabel('Optimization Step')
plt.ylabel('Energy')
plt.title('Energy vs Optimization Step')

In [ ]:
update_theta = Optimizer.x
print(update_theta)

In [ ]:
def UV_function(
    nqbits: int,
    phi: List[float],
    theta: List[float],
    k_targ: int, 
    n_cycles: int = 1,
    n_layers:int =1,
    rotation_gates: List[Gate] = None,
    entangling_gate: Gate = CNOT,
) -> Circuit: #linear entanglement
    
    #This is target state + V
    prog = Program()
    reg = prog.qalloc(nqbits)
    
    
    state = binary_repr(k_targ)
    state_pad = state.zfill(nqbits)
    state_lst = [int(c) for c in state_pad]


    for j in range(min(nqbits, len(state_lst))):
        if int(state_lst[j] == 1):
            prog.apply(X, reg[j])
        
        if int(state_lst[j] == 0):
            prog.apply(I, reg[j])
    
    print("This target state is ", state_pad)
    
    
    
    
    if rotation_gates is None:
        rotation_gates = [RZ]

    n_rotations = len(rotation_gates)

    
    


    ind_phi = 0



    for k in range(n_cycles):
            for i in range(2):
                prog.apply(I, reg[i])
        # Apply rotational gates from qubit 2 to nqbits - 1
            
            for i in range(2, nqbits):
                for rot in rotation_gates:

                    prog.apply(rot(phi[ind_phi]), reg[i])
                    ind_phi += 1
            
            for i in range(2, nqbits - 1):
                prog.apply(CNOT, reg[i], reg[i+1])



    # This is U with update value of theta

    
    
    ind_theta = 0


    for i in range(nqbits):

        for rot in rotation_gates:

            prog.apply(rot(theta[ind_theta]), reg[i])
            ind_theta += 1
    
    for k in range(n_layers):


        for i in range(nqbits - 1):
            prog.apply(CNOT, reg[i], reg[i+1])
            
        for i in range(nqbits):
            for rot in rotation_gates:
                            
                prog.apply(rot(theta[ind_theta]), reg[i])
                ind_theta += 1

    return prog.to_circ()



In [ ]:
n_rotations = 2
nqbits = 4
n_cycles =3 # this is for V
n_layers = 3

phi = [prog.new_var(float, rf"\phi_{{{i}}}") for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_cycles))]
print("Values that will be distribute to all the indicator of phi as applied state are :", phi)
theta = [prog.new_var(float, rf"\theta_{{{i}}}") for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_layers))]

circ_exci_3 = UV_function(4, phi, update_theta, 6, 3, 3, [RX,RZ], CNOT)

circ_exci_3.display()


In [ ]:
n_rotations = 2
nqbits = 4
n_cycles =3 # this is for V
n_layers = 3 # this is for U
phi = [prog.new_var(float, rf"\phi_{{{i}}}") for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_cycles))]
print("Values that will be distribute to all the indicator of phi as applied state are :", phi_0)
U_theta = U_theta(4, update_theta, 13, 3, [RX,RZ], CNOT)
U_theta.display()
theta = [prog.new_var(float, rf"\theta_{{{i}}}") for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_layers))]
circ_exci_3 = UV_function(4, phi, U_theta, 6, 3, 3, [RX,RZ], CNOT)
circ_exci_3.display()

In [ ]:

def opt_funct(circ_funct_exci_3_input, model, qpu, nqbits, energy_circ_exci_3):
    def input_funct(x):
        opt_circ_exci_3 = circ_funct_exci_3_input.bind_variables({k: v for k, v in zip(sorted(circ_funct_exci_3_input.get_variables()), x)})


        stack3 = qpu.submit(opt_circ_exci_3.to_job(observable=model))

        
        energy_exci_3 = stack3.value


        energy_circ_exci_3[method].append(energy_exci_3)


        energy_cost = -1.0*energy_exci_3 
        return energy_cost

    def callback(x):
        opt_circ_ground = circ_funct_ground_input.bind_variables({k: v for k, v in zip(sorted(circ_funct_ground_input.get_variables()), x)})

        
        opt_circ_exci_3 = circ_funct_exci_3_input.bind_variables({k: v for k, v in zip(sorted(circ_funct_exci_3_input.get_variables()), x)})


        stack3 = qpu.submit(opt_circ_exci_3.to_job(observable=model))

        
        energy_exci_3 = stack3.value


        energy_circ_exci_3[method].append(energy_exci_3)

    return input_funct, callback

# Initialize the QPU
qpu = get_default_qpu()

# Set the method
method = "BFGS"



# Initialize energy dictionaries

energy_circ_exci_3 = {method: []}




input_funct,callback = opt_funct(circ_exci_3, model, qpu, nqbits, energy_circ_exci_3)
options = {"disp": True, "maxiter": 500, "gtol": 1e-4}
phi_0 = [np.random.uniform(0, 2*np.pi) for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_cycles))]

Optimizer = scipy.optimize.minimize(input_funct, x0=phi_0, method = "BFGS", callback=callback)